### Load libraries & config

In [ ]:
from datetime import date, datetime, timedelta
import girder_client as gc
import json
import os
import pandas as pd
from urllib.parse import quote
import random
import re
import string
import tzlocal
from urllib.request import urlopen

with open("config.json", "r") as fp:
    config=json.load(fp)
    
def noResponseMessage(user, activity):
    """
    Parameters
    ----------
    user: dictionary
       firstName: string
       lastName: string
    
    activity: string
    
    Return
    ------
    responseMessage: string
    """
    return(
        "\t No {} responses".format(
            activity
        )
    )

### Connect to Girder

In [ ]:
girder_connection = gc.GirderClient(
    apiUrl="{}/api/v1/".format(
        config["girder-production"]["host"]
    )
)
girder_connection.authenticate(
    username=config["girder-production"]["user"],
    password=config["girder-production"]["password"]
)

In [ ]:
activity_sets = girder_connection.get(
    "folder?parentType=collection&parentId={}".format(
        girder_connection.get(
            "collection?name=Volumes"
        )[0]["_id"]
    )
)
ema_parent_users = [
    activity_set for activity_set in activity_sets if
    activity_set['name']=="EMA: Parent"
][0]["meta"]["members"]["users"]

In [ ]:
response_table = {}
for user in ema_parent_users:
    user_id = girder_connection.getUser(user)
    if pd.to_datetime(
        user_id["created"]
    ) >= pd.Timestamp(
        2018,
        12,
        21
    ): # Mindlogger Healthy Brain Network beta test 1 start date
        response_table[user_id["_id"]] = {
            "name": "{} {}".format(
                user_id["firstName"],
                user_id["lastName"]
            ),
            "created": user_id["created"][:10]
        }
        try:
            responses = girder_connection.get(
                "folder?parentType=user&parentId={}&name={}".format(
                    user,
                    "Responses"
                )
            )
            if len(responses):
                for response_folder in responses:
                    try:
                        ema_responses = girder_connection.get(
                            "folder?parentType=folder&parentId={}&name={}".format(
                                response_folder["_id"],
                                quote("EMA: Parent")
                            )
                        )
                        if len(ema_responses):
                            for ema_response_folder in ema_responses:
                                ema_response = girder_connection.get(
                                    "item?folderId={}&limit=5000".format(
                                        ema_response_folder["_id"]
                                    )
                                )
                                for response in ema_response:
                                    if response[
                                        'meta'
                                    ]['activity']['name'] in response_table[
                                        user_id["_id"]
                                    ]:
                                        response_table[
                                            user_id["_id"]
                                        ][
                                            response['meta']['activity']['name']
                                        ].append(response)
                                    else:
                                        response_table[
                                            user_id["_id"]
                                        ][
                                            response['meta']['activity']['name']
                                        ] = [response]
                    except:
                        pass
        except:
            pass
response_table = pd.DataFrame(response_table).T.copy()
response_table.sort_values('created', inplace=True)

In [ ]:
report = ""
report += "As of {}:\n\n".format(
    datetime.now().strftime(
        "%Y-%m-%d %I:%M%P"
    )
)
for index, row in response_table.iterrows():
    report += "{}, created {}".format(
        row['name'],
        row['created']
    )
    startDate=date(
        *[
            int(n) for n in row['created'].split(
                '-'
            )
        ]
    )
    userResponses = {
        "Medications": {
            "completed": len(
                row["Medications"]
            ),
            "assigned": 1,
            "datetimes": [
                pd.to_datetime(
                    response["meta"]["responseTime"],
                    unit="ms"
                ) for response in row["Medications"]
            ]
        } if type(row["Medications"])==list else {
            "completed": 0,
            "assigned": 1,
            "datetimes": []
        },
    } if "Medications" in list(response_table.columns) else {}
    for col in [
        col for col in list(response_table.columns) if col not in [
            "name",
            "created",
            "Medications"
        ]
    ]:
        completed = {}
        if type(row[col])==list:
            for response in row[col]:
                d = str(pd.to_datetime(
                    response["meta"]["responseTime"],
                    unit="ms"
                ).strftime(
                    "%Y-%m-%d %I:%M%P"
                ))
                completed[d] = completed[d] + 1 if d in completed else 1
        userResponses[col] = {
            "completed":len(completed.keys()),
            "assigned": (
                (date.today() - startDate)
            ).days,
            "datetimes":["{}{}".format(
                k,
                " (×{})".format(
                    completed[k]
                ) if completed[k] > 1 else ""
            ) for k in completed.keys()]
        } if type(row[col])==list else {
            "completed": 0,
            "assigned": (
                (date.today() - startDate)
            ).days,
            "datetimes": []
        }
    userResponses = pd.DataFrame(userResponses).T.copy()
    for userRowIndex, userRow in userResponses.iterrows():
        report += "\n\t{}: {}/{}{}".format(
            userRow.name,
            userRow["completed"],
            userRow["assigned"],
            "\n\t\t{}".format(
                "\n\t\t".join([
                    d.strftime(
                        "%Y-%m-%d %I:%M%P"
                    ) if not type(d)==str else d for d in userRow["datetimes"]
                ])
            ) if len(userRow["datetimes"]) else ""
        )
    report += "\n\n"

In [ ]:
with open(
    "reports/{}".format(
        datetime.now().strftime(
            "%Y.%m.%d.%H.%M.txt"
        )
    ),
    "w",
    encoding="utf-8"
) as fp:
    fp.write(report)